In [18]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
API_KEY = userdata.get('avalai_api')

In [21]:
!pip install -q google-generativeai python-docx langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [27]:
!pip install -q json5

In [ ]:
!gdown "14ub55iNBBbZyedB7C1U6y-tC1S_1RRcf"
!unzip Data.zip

Adaptive Chunking

In [57]:
import google.generativeai as genai
from docx import Document
import re
import json # Import the json module


genai.configure(api_key=api_key)

# Initialize the Gemini model for classification and token counting
CLASSIFICATION_MODEL_NAME = 'gemini-2.5-flash'
classification_model = genai.GenerativeModel(CLASSIFICATION_MODEL_NAME)

# Function to extract text from DOCX
def extract_text_from_docx(docx_path):
    """Extracts all text from a DOCX file."""
    try:
        document = Document(docx_path)
        text = []
        for paragraph in document.paragraphs:
            text.append(paragraph.text)
        return "\n".join(text)
    except Exception as e:
        print(f"Error extracting text from DOCX: {e}")
        return ""

# Token counting function using Gemini's model
def count_tokens(text):
    """Returns the number of tokens in the input text using Gemini's model."""
    try:
        # Use the model's built-in token counting method
        response = classification_model.count_tokens(text)
        # print(f"DEBUG: Token count for '{text[:50]}...' (API): {response.total_tokens}")
        return response.total_tokens
    except Exception as e:
        print(f"DEBUG: Error counting tokens with Gemini API: {e}. Using fallback estimate.")
        # Fallback to a simple character-based estimate if API call fails
        estimated_tokens = len(text) // 4 # Rough estimate: 1 token ~ 4 characters
        # print(f"DEBUG: Token count for '{text[:50]}...' (Fallback): {estimated_tokens}")
        return estimated_tokens

# Function to chunk text while respecting token limit and sentence boundaries
def chunk_text_by_tokens(text, max_tokens=500):
    """Chunks the text into chunks of sentences while ensuring each chunk doesn't exceed the max token limit."""
    # A more robust sentence split using regex to handle various punctuation and spaces
    # Ensures that the delimiter (e.g., '.', '!', '?') is included at the end of the sentence
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Add back the delimiter if it was removed by split, or ensure it's there
    processed_sentences = []
    for s in sentences:
        s_stripped = s.strip()
        if s_stripped: # Ensure sentence is not empty
            # If the sentence doesn't end with a common punctuation, add a period for consistency
            if not re.search(r'[.!?]$', s_stripped):
                s_stripped += '.'
            processed_sentences.append(s_stripped)

    chunks = []
    current_chunk_sentences = []
    current_token_count = 0
    sentence_ids = []

    for idx, sentence in enumerate(processed_sentences):
        # Count tokens for the current sentence
        sentence_token_count = count_tokens(sentence)

        # print(f"DEBUG: Processing sentence {idx+1}: '{sentence[:50]}...' (Tokens: {sentence_token_count})")
        # print(f"DEBUG: Current chunk tokens: {current_token_count}, Max tokens: {max_tokens}")

        # If adding the sentence would exceed the limit, finalize the current chunk
        # This condition ensures a new chunk is created if the current one is too large,
        # or if the very first sentence itself is larger than max_tokens.
        if current_token_count + sentence_token_count > max_tokens and current_chunk_sentences:
            # Finalize the current chunk
            chunks.append({
                'chunk_id': len(chunks) + 1,
                'tag': 'Public',  # Placeholder tag, will be updated later
                'content': ' '.join(current_chunk_sentences).strip(),
                'source_sentence_ids': sentence_ids
            })
            # print(f"DEBUG: Chunk {len(chunks)} finalized. Starting new chunk.")
            # Start a new chunk with the current sentence
            current_chunk_sentences = [sentence]
            sentence_ids = [idx + 1]  # sentence IDs start from 1
            current_token_count = sentence_token_count
        elif sentence_token_count > max_tokens:
            # Handle case where a single sentence is larger than max_tokens
            # It will form its own chunk. This might lead to a chunk exceeding max_tokens,
            # but it respects sentence boundaries as requested.
            chunks.append({
                'chunk_id': len(chunks) + 1,
                'tag': 'Public',
                'content': sentence,
                'source_sentence_ids': [idx + 1]
            })
            print(f"DEBUG: Sentence {idx+1} (Tokens: {sentence_token_count}) > Max tokens. Creating single-sentence chunk.")
            current_chunk_sentences = [] # Reset for next sentence
            sentence_ids = []
            current_token_count = 0
        else:
            # Add the sentence to the current chunk
            current_chunk_sentences.append(sentence)
            sentence_ids.append(idx + 1)
            current_token_count += sentence_token_count

        # print(f"DEBUG: After processing sentence {idx+1}: Current chunk tokens: {current_token_count}")

    # Add the last chunk if it contains content
    if current_chunk_sentences:
        chunks.append({
            'chunk_id': len(chunks) + 1,
            'tag': 'Public',  # Placeholder tag, will be updated later
            'content': ' '.join(current_chunk_sentences).strip(),
            'source_sentence_ids': sentence_ids
        })
        # print(f"DEBUG: Final chunk {len(chunks)} added.")

    return chunks

# Function to classify each chunk using Gemini
def classify_chunk(chunk_content):
    """Classifies a chunk using the Gemini API."""
    prompt = (
        "Classify the following text into one of these categories: 'Public', 'Sensitive', or 'Confidential'. "
        "Provide only the category name as your response, without any additional text or explanation.\n\n"
        f"Text: \"\"\"{chunk_content}\"\"\""
    )

    try:
        response = classification_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=10, # Keep output short for classification
                temperature=0.0,      # Aim for deterministic classification
            )
        )
        # Extract the text from the response
        if response.candidates:
            # Access the text directly from the first part of the first candidate
            tag = response.candidates[0].content.parts[0].text.strip()
            # Basic validation to ensure the tag is one of the expected values
            if tag in ['Public', 'Sensitive', 'Confidential']:
                return tag
            else:
                print(f"Warning: Unexpected classification result: '{tag}'. Defaulting to 'Public'.")
                return 'Public'
        else:
            print("Warning: No candidates found in Gemini API response. Defaulting to 'Public'.")
            return 'Public'
    except Exception as e:
        print(f"Error classifying chunk with Gemini API: {e}. Defaulting to 'Public'.")
        return 'Public'

# Function to save chunked data to a JSON file
def save_chunks_to_json(chunks_data, output_filename="chunked_data.json"):
    """Saves the list of chunk dictionaries to a JSON file."""
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(chunks_data, f, indent=4, ensure_ascii=False)
        print(f"Chunked data successfully saved to {output_filename}")
    except Exception as e:
        print(f"Error saving chunked data to JSON file: {e}")

# Main function to process DOCX, chunk the text, and tag the chunks
def process_document(docx_path, output_filename, max_tokens=500):
    """Processes a DOCX file, chunking it and classifying each chunk."""
    # Step 1: Extract text from the DOCX file
    text = extract_text_from_docx(docx_path)
    if not text:
        print("No text extracted from the document. Exiting.")
        return []

    # Step 2: Chunk the text into manageable parts
    chunks = chunk_text_by_tokens(text, max_tokens)

    # Step 3: Classify each chunk and assign the appropriate tag
    chunk_metadata = []
    for chunk in chunks:
        print(f"Classifying chunk {chunk['chunk_id']}...")
        tag = classify_chunk(chunk['content'])
        chunk['content'] = f'[{tag.upper()}]:' + chunk['content']
        chunk['tag'] = tag  # Update the chunk's tag
        chunk_metadata.append(chunk)

    save_chunks_to_json(chunk_metadata,output_filename= output_filename)
    # return chunk_metadata


In [58]:
document_path = "/content/Data/Docs/Customer Service/1.docx"
process_document(document_path, output_filename="chunked_data_tag.json")
# print(f"-------ch: {chunk_metadata}------------")

Classifying chunk 1...
Error classifying chunk with Gemini API: list index out of range. Defaulting to 'Public'.
Classifying chunk 2...
Error classifying chunk with Gemini API: list index out of range. Defaulting to 'Public'.
Classifying chunk 3...
Error classifying chunk with Gemini API: list index out of range. Defaulting to 'Public'.
Chunked data successfully saved to chunked_data_tag.json


In [ ]:
document_path = "/content/Data/Docs/Customer Service/1.docx"
doc = extract_text_from_docx(document_path)
chunks = chunk_text_by_tokens(doc)
print(chunks)

In [12]:
len(chunks)

3

In [16]:
chunks[0]['content']

'Understanding Customer Service Excellence: A Comprehensive Guide\n\n1. Introduction to Customer Service Excellence\n Customer service excellence is the practice of providing outstanding support and service to customers at every point of contact. It involves understanding customer needs, exceeding their expectations, and building lasting relationships. Exceptional customer service can differentiate a business from its competitors and drive customer loyalty and retention. 2. Key Principles of Customer Service Excellence\n The key principles of customer service excellence include empathy, responsiveness, reliability, and communication. Empathy ensures that customers feel heard and understood, while responsiveness demonstrates a commitment to meeting their needs. Reliability builds trust, and effective communication ensures that customers are always informed and engaged. 3. The Importance of Customer Satisfaction\n Customer satisfaction is crucial to the success of any business. Satisfied

In [17]:
# Step 4: Execute the process
chunked_data = process_document(document_path, max_tokens=500)

# Step 5: Save the chunked data to a JSON file
if chunked_data:
    save_chunks_to_json(chunked_data, output_filename='chunked_output.json')

Classifying chunk 1...
Error classifying chunk with Gemini API: list index out of range. Defaulting to 'Public'.
Classifying chunk 2...
Error classifying chunk with Gemini API: list index out of range. Defaulting to 'Public'.
Classifying chunk 3...
Error classifying chunk with Gemini API: list index out of range. Defaulting to 'Public'.
Chunked data successfully saved to chunked_output.json


Privacy Gateway

In [24]:
import json, re
from typing import Literal, List, Dict

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# ─────────── 0.  MODEL HANDLES ──────────────────────────────
def make_llm(temp: float) -> ChatOpenAI:
    return ChatOpenAI(
        base_url = "https://api.avalai.ir/v1",
        model="gpt-4o",
        api_key=API_KEY,
        temperature=temp,
    )

LLM_PRECISE = make_llm(0.0)   # Phase A
LLM_DEEP    = make_llm(0.7)   # Phase B

MAX_TOKENS = 512

# ─────────── 1-A.  PRECISION PROMPT  (Phase A) ─────────────
PRECISION_TMPL = """
You are the Privacy Gate.

Rewrite the text so that its vagueness matches ε = {epsilon}.

Guidelines
• ε = 1.0  → keep almost all specifics; redact only obvious secrets
• ε = 0.7  → generalise a little
• ε = 0.5  → remove/blur most specifics
• ε = 0.3  → keep only broad actions
• ε = 0.1  → bare outline

Do NOT remove the core meaning.

Label: {label}
Original text:
\"\"\"{chunk}\"\"\"

Return JSON only:
{{"rewritten": "<sanitised>"}}"""

# ─────────── 1-B.  DEEP-OBFUSCATE PROMPT (Phase B) ─────────
DEEP_TMPL = """
You are the Privacy Gate – deep-obfuscation mode.

Take the input text and rewrite it so it is **even vaguer** than before:
• shorten sentences,
• replace any remaining specifics with generic terms,
• rephrase with **different wording** than the previous version,
• keep overall intent.

Return JSON only:
{{"rewritten": "<even more vague text>"}}"""

# ─────────── 2.  LOW-LEVEL CALL HELPERS ─────────────────────
_JSON_RE = re.compile(r"\{.*\}", re.DOTALL)

def _call(llm: ChatOpenAI, prompt: str) -> str:
    resp = llm(
        [SystemMessage(content="You are a privacy-focused assistant."),
         HumanMessage(content=prompt)],
        max_tokens=MAX_TOKENS,
    ).content.strip()
    m = _JSON_RE.search(resp)
    if not m:
        raise ValueError(f"Missing JSON:\n{resp}")
    obj = json.loads(m.group(0))
    if "rewritten" not in obj:
        raise ValueError(f"No 'rewritten' key:\n{obj}")
    return obj["rewritten"]

# ─────────── 3-A.  single-pass sanitiser (Phase A) ─────────
def sanitise_once(chunk: str, label: str, ε: float) -> str:
    prompt = PRECISION_TMPL.format(
        label=label, epsilon=ε, chunk=chunk.replace('"', '\\"')
    )
    return _call(LLM_PRECISE, prompt)

# ─────────── 3-B.  deep-obfuscate step (Phase B) ───────────
def deep_obfuscate(prev: str) -> str:
    prompt = DEEP_TMPL.format(chunk=prev.replace('"', '\\"'))
    return _call(LLM_DEEP, prompt)

# ─────────── 4.  public pipeline ───────────────────────────
def privacy_gate_pipeline(
    text: str,
    label: Literal["PUBLIC", "SENSITIVE", "CONFIDENTIAL"],
    eps_schedule: List[float] = (1.0, 0.7, 0.5, 0.3, 0.1),
    deep_rounds: int = 4,
) -> Dict[float, List[str]]:
    """
    Phase A: run once for every ε in eps_schedule
    Phase B: take ε=min(eps_schedule) output, run 'deep_rounds' extra passes
    Returns {ε: [v1, v2, …]}  (ε>min → one element; ε_min → deep_rounds+1 elems)
    """
    results = {}
    current = text
    for ε in eps_schedule:
        current = sanitise_once(current, label, ε)
        results[ε] = [current]

    ε_min = min(eps_schedule)
    for _ in range(deep_rounds):
        current = deep_obfuscate(current)
        results[ε_min].append(current)

    return results

In [44]:
import os
import json
import json5

# questions_root = "/kaggle/input/dataset-ldp/Data_LDP/Attack Question"
questions_root = '/content/Data/Answer Questions '
results        = []                                      # collect everything here

output_file = '/content/processed_results.json'
with open(output_file, 'w') as json_out:
    json.dump([], json_out)  # Initialize the file with an empty list

    for folder in sorted(os.listdir(questions_root)):
        folder_path = os.path.join(questions_root, folder)
        if not os.path.isdir(folder_path):
            continue

        for i in range(1, 11):  # 1 … 10
            json_path = os.path.join(folder_path, f"{i}.json")
            if not os.path.exists(json_path):
                continue

            with open(json_path, "r") as f:
                qlist = json5.load(f)

            for item in qlist:
                # Call the privacy_gate_pipeline function
                label = item['label']
                text_label = item['answer']
                # data = {'resp', text_label}
                # json_data = json.dumps(data)
                text_label = '[resp]:'++ text_label
                print(f"-----------text_label:{text_label} - label:{label}-----------")
                results = privacy_gate_pipeline(text=text_label, label=label.upper() )

                # Prepare the result to be written to the file
                processed_item = {'original_item': item, 'pipeline_results': results}

                # Open the file again in append mode and write the results immediately
                # with open(output_file, 'r+') as json_out:
                #     existing_data = json.load(json_out)  # Load the existing data
                #     existing_data.append(processed_item)  # Append the new result
                #     json_out.seek(0)  # Move cursor to the beginning
                #     json.dump(existing_data, json_out, indent=4)  # Write the updated data back

                # (optional) show progress in the Kaggle console
                print(f"✓ results : {results}")

print(f"✓ Processed results saved to {output_file}")

-----------text_label:respRegularly tracking these metrics helps to identify areas for improvement and ensures that the practices of customer service are in alignment with the goals of the business. - labelConfidential-----------
✓ results : {1.0: ['Regularly monitoring these metrics supports identifying improvement opportunities and ensures customer service aligns with business objectives.'], 0.7: ['Regularly reviewing these indicators helps recognize areas for enhancement and maintains alignment with organizational goals.'], 0.5: ['Consistently assessing these factors supports improvement and ensures alignment with objectives.'], 0.3: ['Evaluating aspects helps progress and maintains focus.'], 0.1: ['Assessing supports improvement.', 'Details adjusted, focus shifted, meaning simplified.', 'Things happen, choices made, outcomes follow.', 'Details were altered; events unfolded in unclear ways. Things happened, but specifics faded into obscurity.', 'Details unclear, broad ideas suggeste

KeyboardInterrupt: 